# Demo di Steering LLM (Versione Modulare)

Questo notebook dimostra come usare la libreria `steering_lib` per applicare l'Activation Engineering su GPT-2.
Useremo la nuova architettura modulare con commenti e classi in italiano.

In [1]:
%load_ext autoreload
%autoreload 2

import torch
from steering_lib import ModelloSteerable

In [2]:
# Inizializziamo il modello (GPT-2 Small)
modello = ModelloSteerable(model_name="gpt2", device="cuda") # Cambia in "cuda" se hai una GPU

Caricamento modello gpt2 su cuda...


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

c:\Users\solda\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\solda\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

## 1. Estrazione del Vettore di Steering
Calcoliamo la differenza tra le attivazioni di un prompt positivo e uno negativo a un livello specifico.

In [3]:
prompt_positivo = "I love this movie because"
prompt_negativo = "I hate this movie because"
layer_idx = 6  # GPT-2 ha 12 layer (0-11). I layer centrali sono buoni per concetti semantici.

# Estraiamo il vettore usando il nuovo metodo della classe ModelloSteerable
vettore_steering = modello.estrai_vettore(prompt_positivo, prompt_negativo, layer_idx)
print(f"Vettore calcolato dal layer {layer_idx}. Norma: {vettore_steering.norm().item():.2f}")

Vettore calcolato dal layer 6. Norma: 22.16


## 2. Test dello Steering
Generiamo testo con un prompt neutro, applicando il vettore con diversi moltiplicatori.

In [4]:
prompt_test = "I think that this film is"

print("--- Baseline (Nessuno Steering) ---")
print(modello.genera(prompt_test, moltiplicatore=0))

print("\n--- Steering Positivo (moltiplicatore=5.0) ---")
print(modello.genera(prompt_test, vettore_steering=vettore_steering, layer_idx=layer_idx, moltiplicatore=5.0))

print("\n--- Steering Negativo (moltiplicatore=-5.0) ---")
print(modello.genera(prompt_test, vettore_steering=vettore_steering, layer_idx=layer_idx, moltiplicatore=-5.0))

--- Baseline (Nessuno Steering) ---
I think that this film is a case of making things work, and the only time I was able to do this was in '

--- Steering Positivo (moltiplicatore=5.0) ---
I think that this film is especially easy to do and many divers, at both the unique combination of design and flexibility of diverse development

--- Steering Negativo (moltiplicatore=-5.0) ---
I think that this film is for the in the sub to the child, in the way I am going to call the black in
